In [1]:
import sys
print(sys.version)

3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]


In [2]:
import pandas as pd
print(pd.__version__)

0.25.2


In [3]:
#pd.options.display.max_columns = None
#pd.options.display.float_format = '{:,.2f}'.format
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
csl_df = pd.read_sas('../data/csl/csllinkedbypreg.sas7bdat', format = 'sas7bdat', encoding='iso-8859-1')
csl_df.head()

,Sitenum,MomID,Marital,Education,Insurance,Momage,Parity,Gravidity,Hxlivebirth,Hxmacrosomia,...,Fulldilattime,Placenttime,Inducttime,Ripentime,ROMtime,Epiduraltime,Intraantibtime,USdays,LMPdays,onepregflag
0,41,41-00001,1.0,3.0,1.0,34.0,4.0,5.0,4.0,1.0,...,15.0,4.0,361.0,NaN,300.0,343.0,346.0,103.0,271.0,2.0
1,41,41-00001,1.0,3.0,1.0,36.0,5.0,6.0,5.0,2.0,...,18.0,3.0,369.0,NaN,251.0,314.0,NaN,128.0,278.0,2.0
2,41,41-00002,1.0,3.0,1.0,37.0,1.0,3.0,1.0,1.0,...,NaN,1.0,NaN,NaN,1.0,4.0,NaN,128.0,268.0,1.0
3,41,41-00003,1.0,3.0,1.0,36.0,1.0,4.0,1.0,1.0,...,69.0,4.0,382.0,NaN,369.0,364.0,NaN,133.0,273.0,1.0
4,41,41-00004,1.0,2.0,1.0,34.0,2.0,6.0,2.0,1.0,...,12.0,5.0,339.0,NaN,105.0,116.0,455.0,204.0,274.0,1.0


In [6]:
%%time
# Version of convert_integer stored in util_mwb
from util_mwb import convert_integer 
all_types = convert_integer(csl_df)

CPU times: user 5min 26s, sys: 4min 40s, total: 10min 6s
Wall time: 7min 46s


In [7]:
import pickle
#with open('csl_types.pickle', 'wb') as file:
#    pickle.dump(all_types, file)  # Commented out to avoid overwrite
    
with open('csl_types.pickle', 'rb') as file:
    read_types = pickle.load(file)
    
print( all_types == read_types)
    

True


In [8]:
csl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228438 entries, 0 to 228437
Columns: 779 entries, Sitenum to onepregflag
dtypes: Int16(38), Int32(2), Int8(663), float64(59), object(17)
memory usage: 448.3+ MB


In [9]:
# Original code (copied to util_mwb); left here if optimization needed
def convert_integer (df):
    type_dict = {}
    for col in df.columns:
        intcol_flag = True
        if df[col].dtype == 'float64':
            for val in df[col]:
                if pd.notnull(val) and abs(val - int(val)) > 1e-6:
                    intcol_flag = False
                    break;
            if intcol_flag:
                if df[col].abs().max() < 127:
                    df[col] = df[col].astype('Int8')
                elif df[col].abs().max() < 32767:
                    df[col] = df[col].astype('Int16')
                else:   # assuming no ints greater than 2147483647 
                    df[col] = df[col].astype('Int32') 
        print(f"{col} is {df[col].dtype}")
        type_dict[col] = df[col].dtype
    return type_dict

In [10]:
# Possible attempts to speed inner loop - didn't work
#csl_df.applymap(lambda x: str(int(x)) if abs(x - int(x)) < 1e-6 else str(round(x,2)))
#csl_df['Marital'].apply(lambda x: str(int(x)) if abs(x - int(x)) < 1e-6 else str(round(x,2)))

In [21]:
types = csl_df.dtypes
print(type(types))
types.value_counts()

<class 'pandas.core.series.Series'>


Int8       663
float64     59
Int16       38
object      17
Int32        2
dtype: int64

In [12]:
types[(types != 'object') & (types != float)].index.values

array(['Marital', 'Education', 'Insurance', 'Momage', 'Parity',
       'Gravidity', 'Hxlivebirth', 'Hxmacrosomia', 'Hxpreterm',
       'Hxstillbirth', 'HxnumCS', 'Hxcsection', 'Diabetes', 'ChronicHBP',
       'Hxheartdis', 'Hxrenaldis', 'HxGIdis', 'Hxdepression', 'Hxseizure',
       'Hxthyroid', 'Hxasthma', 'Hxanemia', 'HIV', 'HxSTD', 'Numfetus',
       'OGTT1h', 'ART', 'Smoke', 'Alcohol', 'Druguse', 'ECV', 'Cerclage',
       'Antesteroid', 'Antehospital', 'AnteSTD', 'AnteGBS', 'AnteUTI',
       'Activeherpes', 'Antebleed3', 'Anteanemia', 'Anteasthma',
       'Antethyroid', 'GestHBP', 'Preeclampsia', 'Eclampsia', 'ImposedPE',
       'UnspecHBP', 'AnteCHBP', 'AnteGDM', 'DMControl', 'Anteprevia',
       'Anteabruption', 'Antethrombo', 'Antechorio', 'ThreatenedPB',
       'Incompetence', 'PROM', 'AnteSGA', 'AnteLGA', 'Rhincompat',
       'Oligo', 'Polyhydramnios', 'Antefetdistress', 'CPD', 'TTTwin',
       'Antefetaldth', 'AdmSBP', 'AdmDBP', 'AdmEFW', 'Admpresent',
       'AdmBishop', 'GA

In [13]:
csl_df.isnull().sum()

Sitenum                0
MomID                  0
Marital                1
Education              1
Insurance              1
                   ...  
Epiduraltime      124528
Intraantibtime    181958
USdays            163192
LMPdays           121799
onepregflag            0
Length: 779, dtype: int64

In [14]:
csl_df.describe(include='all')

,Sitenum,MomID,Marital,Education,Insurance,Momage,Parity,Gravidity,Hxlivebirth,Hxmacrosomia,...,Fulldilattime,Placenttime,Inducttime,Ripentime,ROMtime,Epiduraltime,Intraantibtime,USdays,LMPdays,onepregflag
count,228438,228438,228437.000000,228437.000000,228437.000000,228115.000000,228438.000000,226720.000000,160448.000000,228438.000000,...,145591.000000,192847.000000,65530.000000,10917.000000,76061.000000,103910.000000,46480.000000,65246.000000,106639.000000,228438.000000
unique,12,208695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,41,45-01637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,50320,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.970184,6.432855,2.081563,27.627946,1.139894,2.667665,1.084626,4.170786,...,72.767960,8.262643,567.211323,958.151598,437.929885,308.309922,381.678442,145.074165,269.954848,1.168632
std,NaN,NaN,1.457665,2.865649,2.083136,6.196163,1.367558,1.826788,1.280143,3.634488,...,211.966159,71.324187,535.209984,613.710233,659.183549,255.902863,606.139353,59.235567,18.760556,0.374428
min,NaN,NaN,1.000000,1.000000,1.000000,11.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,NaN,NaN,1.000000,3.000000,1.000000,23.000000,0.000000,1.000000,0.000000,1.000000,...,13.000000,2.000000,233.000000,559.000000,106.000000,134.000000,105.000000,123.000000,266.000000,1.000000
50%,NaN,NaN,1.000000,8.000000,1.000000,27.000000,1.000000,2.000000,1.000000,1.000000,...,32.000000,4.000000,414.000000,846.000000,281.000000,252.000000,228.000000,143.000000,274.000000,1.000000
75%,NaN,NaN,3.000000,8.000000,2.000000,32.000000,2.000000,3.000000,2.000000,8.000000,...,79.000000,6.000000,715.000000,1213.000000,553.000000,419.000000,459.000000,193.000000,280.000000,1.000000


In [15]:
#csl_df.to_csv('/home/mbopf/MFMDatasets/CSL_StudyItems/mbopf/csl_all.csv')

In [16]:
# Clear some memory
#import gc
#del csl_df
#gc.collect()

In [26]:
import numpy as np
import pandas as pd
np.random.seed(10)
df = pd.DataFrame(np.random.choice([1, 2, 3.3, 5000, 111111, np.NaN], (3,9)), 
                  columns=[f'col{i}' for i in range(9)])
df

,col0,col1,col2,col3,col4,col5,col6,col7,col8
0,2.0,NaN,111111.0,1.0,2.0,5000.0,111111.0,2.0,NaN
1,1.0,NaN,2.0,3.3,1.0,2.0,1.0,3.3,1.0
2,111111.0,5000.0,1.0,111111.0,5000.0,1.0,5000.0,3.3,2.0


In [24]:
import util_mwb
util_mwb.convert_integer(df)

{'col0': Int32Dtype(),
 'col1': Int16Dtype(),
 'col2': Int32Dtype(),
 'col3': dtype('float64'),
 'col4': Int16Dtype(),
 'col5': Int16Dtype(),
 'col6': Int32Dtype(),
 'col7': dtype('float64'),
 'col8': Int8Dtype()}

In [25]:
df.max(axis=1)

0    111111.0
1         3.3
2    111111.0
dtype: float64

In [20]:
s = pd.cut(df.max(), bins=[0, 127, 32767, 2147483647], labels=['Int8', 'Int16', 'Int32'])
print(s)
s = s.where(df.dtypes=='float')  # So we only cast the previously float cols 
print(s)

#for idx, gp in s.groupby(s):
#    df.loc[:, gp.index] = df.loc[:, gp.index].astype(idx)

col0    Int32
col1    Int16
col2    Int32
col3    Int32
col4    Int16
col5    Int16
col6    Int32
col7     Int8
col8     Int8
dtype: category
Categories (3, object): [Int8 < Int16 < Int32]
col0      NaN
col1      NaN
col2      NaN
col3    Int32
col4      NaN
col5      NaN
col6      NaN
col7     Int8
col8      NaN
dtype: category
Categories (3, object): [Int8 < Int16 < Int32]
